# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [14]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [15]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,292.27,72,100,9.83,PN,1720385199
1,1,nova sintra,14.8667,-24.7167,298.65,80,43,2.93,CV,1720385200
2,2,susuman,62.7833,148.1667,283.65,76,22,2.06,RU,1720385202
3,3,vadso,70.0744,29.7487,287.64,88,75,1.54,NO,1720385203
4,4,port-aux-francais,-49.3500,70.2167,275.51,87,98,12.07,TF,1720385204


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", "Lat", geo=True, size="Humidity", color="Humidity", alpha=0.5,
    tiles="OSM", frame_width=800, frame_height=600, hover_cols=["City", "Country"]
)


# Display the map
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [21]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[
    (city_data_df["Max Temp"] > 200) & (city_data_df["Max Temp"] < 350) &
    (city_data_df["Wind Speed"] < 3) & (city_data_df["Cloudiness"] == 0)
]



# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()


# Display sample data
ideal_weather_df.head()



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
44,44,chadan,51.2833,91.5833,286.21,85,0,1.25,RU,1720385251
94,94,maun,-19.9833,23.4167,284.13,28,0,1.03,BW,1720385314
107,107,saint-francois,46.4154,3.9054,288.84,73,0,1.03,FR,1720385331
126,126,mendoza,-32.8908,-68.8272,280.89,31,0,2.57,AR,1720385119
137,137,new norfolk,-42.7826,147.0587,276.32,97,0,1.18,AU,1720385367


### Step 3: Create a new DataFrame called `hotel_df`.

In [22]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""


# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
44,chadan,RU,51.2833,91.5833,85,
94,maun,BW,-19.9833,23.4167,28,
107,saint-francois,FR,46.4154,3.9054,73,
126,mendoza,AR,-32.8908,-68.8272,31,
137,new norfolk,AU,-42.7826,147.0587,97,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [23]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
chadan - nearest hotel: Жарки
maun - nearest hotel: Center Lodge Conference Center
saint-francois - nearest hotel: Chez Lily
mendoza - nearest hotel: Hotel Eben-Ezer
new norfolk - nearest hotel: Woodbridge on the Derwent
colonia - nearest hotel: Wasserturm Hotel Cologne
tiksi - nearest hotel: Арктика
zonguldak - nearest hotel: No hotel found
sarpol-e zahab - nearest hotel: No hotel found
montefrio - nearest hotel: Monteftur
luanda - nearest hotel: Hotel Forum
gumushacikoy - nearest hotel: No hotel found
kaiapoi - nearest hotel: No hotel found
vernon - nearest hotel: Okanagan Royal Park Inn
portland - nearest hotel: the Hoxton
arraial do cabo - nearest hotel: No hotel found
constantia - nearest hotel: Hotel Dali
thabazimbi - nearest hotel: Maroela Guest Lodge
argo - nearest hotel: Apollon Hotel
siran - nearest hotel: No hotel found
ingeniero jacobacci - nearest hotel: Gran Hotel Argentino
newman - nearest hotel: No hotel found
patterson - nearest hotel: Hampton Inn

,City,Country,Lat,Lng,Humidity,Hotel Name
44,chadan,RU,51.2833,91.5833,85,Жарки
94,maun,BW,-19.9833,23.4167,28,Center Lodge Conference Center
107,saint-francois,FR,46.4154,3.9054,73,Chez Lily
126,mendoza,AR,-32.8908,-68.8272,31,Hotel Eben-Ezer
137,new norfolk,AU,-42.7826,147.0587,97,Woodbridge on the Derwent
144,colonia,DE,50.9333,6.9500,71,Wasserturm Hotel Cologne
156,tiksi,RU,71.6872,128.8694,72,Арктика
172,zonguldak,TR,41.2500,31.8333,73,No hotel found
201,sarpol-e zahab,IR,34.4611,45.8626,27,No hotel found
211,montefrio,ES,37.3231,-4.0090,30,Monteftur


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [24]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng", "Lat", geo=True, size="Humidity", color="Humidity", alpha=0.5,
    tiles="OSM", frame_width=800, frame_height=600, hover_cols=["City", "Country", "Hotel Name"]
)


# Display the map
hotel_map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)